#SoMe Topic Modeling Notebook | Release canvas 1 📖

## Installations and Libraries 💽

In [1]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    # !pip install --upgrade autopep8

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter
import time 

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib as plt 
import pyLDAvis.gensim

#Natural Language Processing (NLP)
import spacy
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

/usr/local/lib/python3.6/dist-packages/_pytest/mark/structures.py:426: DeprecationWarning:

The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.

/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



##Data Cleaning 🧹

In [3]:
# Loading the JSON file 
url_elon = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/elonmusk_followers_english.json'
url_dutchbros = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/dutchbros_followers.json'

df = requests.get(url_elon).json()

# Converting the dataset to pandas DataFrame and renaming the columns 
df = pd.DataFrame(df.values())
df = df.rename(columns={0:'original_tweets'})

#Removing emojies from text
#Refrence 1 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 2 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df['emoji_free_tweets'] = df['original_tweets'].apply(call_emoji_free)

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)

df

,original_tweets,emoji_free_tweets,url_free_tweets
0,This kid will forever be a legend 😂 https://t....,This kid will forever be a legend https://t.co...,This kid will forever be a legend
1,"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!
3,@Bhuvan_Bam ❤️❤️,@Bhuvan_Bam,@Bhuvan_Bam
4,I'm not crying you're crying.\nhttps://t.co/Bc...,I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.
...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...
9942,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu..."
9944,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...


##Topic Modeling ㊙️

###Tokenizing 🕵🏻‍♂

In [0]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')


In [5]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['url_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens

# Timing End
program_end_time = time.time()

# View df
df

,original_tweets,emoji_free_tweets,url_free_tweets,tokens
0,This kid will forever be a legend 😂 https://t....,This kid will forever be a legend https://t.co...,This kid will forever be a legend,"[kid, forever, legend]"
1,"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","[truly, believe, lebrons, mindset,, competitiv..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,"[buttlicker!, prices, lower!!!]"
3,@Bhuvan_Bam ❤️❤️,@Bhuvan_Bam,@Bhuvan_Bam,[@bhuvan_bam]
4,I'm not crying you're crying.\nhttps://t.co/Bc...,I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.,"[crying, you're, crying.]"
...,...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,"[@mirandasleeper, offerings, f-3, closed, prev..."
9942,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,"[defining, segment, “the, dance.”]"
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","[@frank_miskelly, don’t, it,, love, it!, think..."
9944,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,"[excited, brother, @shufly10, embarks, new, jo..."


In [6]:
# See how long it took
print(program_end_time - program_start_time, "seconds to finish")

2.023756504058838 seconds to finish


###Lemmatization🇬🇧

In [0]:
# Refrence 4 : https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column

# Timing Start
program_start_time = time.time()

# Make tokens a string again
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
# df[['original_tweet', 'lemmas_back_to_text']]

# Timing End
program_end_time = time.time()


In [8]:
#Printing Lemmetization Time
print(program_end_time - program_start_time, "seconds to finish")

100.64755487442017 seconds to finish


In [9]:
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

# Timing End
program_end_time = time.time()

# View those tokens (the 4th column)
df

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationWarning:

invalid escape sequence \$

<input>:22: DeprecationWarning:

invalid escape sequence \w

<input>:23: DeprecationW

,original_tweets,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,This kid will forever be a legend 😂 https://t....,This kid will forever be a legend https://t.co...,This kid will forever be a legend,"[kid, forever, legend]",kid forever legend,"[kid, forever, legend]",kid forever legend,"[kid, forever, legend]"
1,"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","[truly, believe, lebrons, mindset,, competitiv...","truly believe lebrons mindset, competitive fir...","[truly, believe, lebrons, mindset, competitive...",truly believe lebrons mindset competitive fire...,"[truly, believe, lebrons, mindset, competitive..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,"[buttlicker!, prices, lower!!!]",buttlicker! prices lower!!!,"[buttlicker, price, lower]",buttlicker price lower,"[buttlicker, price, lower]"
3,@Bhuvan_Bam ❤️❤️,@Bhuvan_Bam,@Bhuvan_Bam,[@bhuvan_bam],@bhuvan_bam,[@bhuvan_bam],@bhuvan_bam,[bhuvan_bam]
4,I'm not crying you're crying.\nhttps://t.co/Bc...,I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.,"[crying, you're, crying.]",crying you're crying.,"[cry, cry]",cry cry,"[cry, cry]"
...,...,...,...,...,...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,"[@mirandasleeper, offerings, f-3, closed, prev...",@mirandasleeper offerings f-3 closed previous ...,"[@mirandasleeper, offering, f-3, close, previo...",@mirandasleeper offering f-3 close previous of...,"[mirandasleeper, offering, f-3, close, previou..."
9942,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,"[defining, segment, “the, dance.”]",defining segment “the dance.”,"[define, segment, dance]",define segment dance,"[define, segment, dance]"
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","[@frank_miskelly, don’t, it,, love, it!, think...","@frank_miskelly don’t it, love it! think sixth...","[@frank_miskelly, love, think, sixth, old, run...",@frank_miskelly love think sixth old run middl...,"[frank_miskelly, love, think, sixth, old, run,..."
9944,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,"[excited, brother, @shufly10, embarks, new, jo...","excited brother @shufly10 embarks new journey,...","[excited, brother, @shufly10, embark, new, jou...",excited brother @shufly10 embark new journey o...,"[excited, brother, shufly10, embark, new, jour..."


In [10]:
#Printing Tokenization Time
print(program_end_time - program_start_time, "seconds to finish")


0.3945169448852539 seconds to finish


###id2word 📒

In [11]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))

21883


In [12]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

8229


###Corpus Object & Generating Model 0 Topics 📚

In [0]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

In [14]:
# Timing Start
program_start_time = time.time()

# Instantiating a LDA model 
model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

# Timing End
program_end_time = time.time()


Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa

In [15]:
#Printing First Model Time
print(program_end_time - program_start_time, "seconds to finish")


35.32077360153198 seconds to finish


In [0]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]


In [0]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]


In [18]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
know good think go follow love new retweet people let

------ Topic 1 ------
people time come day need work new thank bitcoin say

------ Topic 2 ------
day year think realdonaldtrump people vote time today state right

------ Topic 3 ------
time obama people follow bitcoin tweet retweet good video year

------ Topic 4 ------
know people obamagate time man good look say need coronavirus



In [19]:
# Compute Perplexity
 # a measure of how good the model is. lower the better
print('\nPerplexity: ', model.log_perplexity(corpus)) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=model, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_0 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_0)


Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa


Perplexity:  -8.457885517026577

Coherence Score:  0.15413898670481424


### Topic Distance Visualization 📈

In [20]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.009429 -0.035980       1        1  24.019552
0      0.069673 -0.012692       2        1  21.695314
4     -0.028481 -0.074902       3        1  21.061424
2     -0.087916  0.059188       4        1  16.644337
3      0.056152  0.064385       5        1  16.579365, topic_info=           Term        Freq       Total Category  logprob  loglift
36         know  439.000000  439.000000  Default  30.0000  30.0000
791     retweet  199.000000  199.000000  Default  29.0000  29.0000
802   obamagate  184.000000  184.000000  Default  28.0000  28.0000
2343   giveaway  137.000000  137.000000  Default  27.0000  27.0000
63          day  427.000000  427.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
123        come   45.825100  296.316620   Topic5  -5.8332  -0.0696
223       watch   37.910355  173.616226   Topic5  -6.0228   0.2754
330          go   43.006138  373.913574   Topic5  -5.8967  -0.3657
216        help   37.395912  189.333191   Topic5  -6.0365   0.1751
572       state   36.390686  213.578934   Topic5  -6.0637   0.0273

[439 rows x 6 columns], token_table=      Topic      Freq Term
term                      
784       1  0.080054    +
784       2  0.290196    +
784       3  0.060041    +
784       4  0.100068    +
784       5  0.460311    +
...     ...       ...  ...
7431      4  0.923045  ہیں
7411      3  0.807358   ہے
7411      4  0.134560   ہے
4552      1  0.848770    ★
4552      3  0.077161    ★

[1078 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 3, 4])

###Imrovements 🔧

####Enhanced Model 1 🦾


In [21]:
# Let's start with parameter tuning for the LDA model and,
# Increase number of topics to be used later to get to 
# Optimal number of distinct topics

# Timing Start
program_start_time = time.time()

model_1 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=20,
                       random_state=100,
                       chunksize=100,
                       passes=10,
                       per_word_topics=True)

# Timing End
program_end_time = time.time()


Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa

In [22]:
#Printing First Model Time
print(program_end_time - program_start_time, "seconds to finish")


41.72494435310364 seconds to finish


In [23]:
# Filtering for words 
words_1 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1.print_topics()]

# Create Topics
topics_1 = [' '.join(t[0:10]) for t in words_1]

# Getting the topics
for id, t in enumerate(topics_1): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
go come realdonaldtrump trump money president guy lead to pay

------ Topic 1 ------
look 4 + word post lie sign announce month comment

------ Topic 2 ------
think join check start world family lose person open job

------ Topic 3 ------
follow 2 tweet hour good thank time retweet giveaway country

------ Topic 4 ------
need help right big work wait allow world mean america

------ Topic 5 ------
amazon project mind gift like national important pandemic bear vaccine

------ Topic 6 ------
day life happy stay ago believe short birthday good actually

------ Topic 7 ------
people crossover make bad game feel change 24 walk today

------ Topic 8 ------
thing 100 home read health tell learn give business want

------ Topic 9 ------
obamagate great rt video break twitter china youtube lot story

------ Topic 10 ------
live die million blah place people plan state 2020 lockdown

------ Topic 11 ------
friend end way covid-19 x force dead 7 low inbox

------ Topic 12 --

In [24]:
# Compute Perplexity
 # a measure of how good the model is. lower the better
print('\nPerplexity: ', model_1.log_perplexity(corpus)) 

# Compute Coherence Score
coherence_model_1 = CoherenceModel(model=model_1, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1 = coherence_model_1.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1)


Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa


Perplexity:  -9.696530535673766

Coherence Score:  0.4094135382787988


In [25]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5     -0.005928  0.042664       1        1  6.963137
18     0.236208  0.063427       2        1  6.131260
3      0.099968 -0.143674       3        1  5.846698
15    -0.141607  0.092653       4        1  5.735518
0     -0.106909 -0.060356       5        1  5.645209
16     0.039856 -0.022874       6        1  5.188169
14     0.025307 -0.209037       7        1  5.162463
7      0.217715  0.074384       8        1  5.051474
4     -0.143392  0.161224       9        1  4.949738
13    -0.130798  0.131231      10        1  4.749745
8      0.089145  0.034537      11        1  4.631353
2     -0.031260  0.002523      12        1  4.628204
9      0.007423 -0.072358      13        1  4.604227
10     0.087217  0.078000      14        1  4.592025
12     0.065841  0.059259      15        1  4.490739
17    -0.095303 -0.129939      16        1  4.484546
6     -0.079182 -0.197020      17        1  4.395856
19    -0.046925  0.090173      18        1  4.375450
11    -0.039477 -0.009512      19        1  4.234931
1     -0.047900  0.014697      20        1  4.139262, topic_info=        Term        Freq       Total Category  logprob  loglift
36      know  438.000000  438.000000  Default  30.0000  30.0000
605      let  389.000000  389.000000  Default  29.0000  29.0000
116    think  361.000000  361.000000  Default  28.0000  28.0000
330       go  485.000000  485.000000  Default  27.0000  27.0000
565  bitcoin  333.000000  333.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
719   league   22.035622   22.893288  Topic20  -5.1777   3.1465
600     call   58.174980   72.586952  Topic20  -4.2069   2.9633
587     hope   67.452095   88.926949  Topic20  -4.0590   2.9083
388     post   85.545036  124.898804  Topic20  -3.8214   2.8062
478    month   74.955696  121.338196  Topic20  -3.9535   2.7030

[721 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
784      20  0.990747          +
72        2  0.867728          1
72       13  0.128871          1
788       3  0.992420         10
139      11  0.995359        100
...     ...       ...        ...
444       2  0.996134       year
157      11  0.984259        yes
1049      2  0.978077  yesterday
517      13  0.988240    youtube
4552     10  0.980816          ★

[723 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 19, 4, 16, 1, 17, 15, 8, 5, 14, 9, 3, 10, 11, 13, 18, 7, 20, 12, 2])

####Model 2 🕑

In [0]:
#Let's change number of topic until so we can get to an oprimal number of topics
program_start_time = time.time()

model_2 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=30,
                       random_state=100,
                       chunksize=100,
                       passes=50,
                       minimum_probability=0.2, 
                       iterations=100)

program_end_time = time.time()


Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa

In [0]:
print(program_end_time - program_start_time, "seconds to finish")


In [0]:
# Filtering for words 
words_2 = [re.findall(r'"([^"]*)"',t[1]) for t in model_2.print_topics()]

# Create Topics
topics_2 = [' '.join(t[0:10]) for t in words_2]

# Getting the topics
for id, t in enumerate(topics_2): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


In [0]:
# Compute Perplexity
 # a measure of how good the model is. lower the better
print('\nPerplexity: ', model_2.log_perplexity(corpus)) 

# Compute Coherence Score
coherence_model_2 = CoherenceModel(model=model_2, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_2 = coherence_model_2.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_2)


In [0]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_2, corpus, id2word)
